# ReAct 模式实验：Reasoning + Acting

## 📚 什么是 ReAct？

**ReAct (Reasoning + Acting)** = 推理 + 行动，让模型在思考和执行工具调用之间交替。

### 核心流程

```
Thought（思考）→ Action（行动）→ Observation（观察）→ [循环] → Answer（答案）
```

### vs CoT

| 维度 | CoT | ReAct |
|-----|-----|-------|
| 能力 | 仅推理 | 推理+工具调用 |
| 外部信息 | ❌ | ✅ |
| 主要失败 | 推理错误 | Action Loop、工具幻觉 |

## 🎯 本实验内容

1. **成功示例**：正常的 ReAct 流程
2. **失败模式1**：Action Loop（无限循环）
3. **失败模式2**：Thought 碎片化
4. **失败模式3**：工具幻觉
5. **失败模式4**：过早终止

## 🤖 测试模型

使用 **Qwen3 8B** 模型

In [6]:
# === 环境准备 ===
import subprocess
import time
import requests
import json
import re

# 安装并启动 Ollama
print("⬇️ 正在安装 Ollama...")
subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True, check=False)
print("✅ Ollama 安装完成")

print("🚀 正在后台启动 Ollama 服务...")
ollama_server = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
print("✅ Ollama 服务已运行")

# 下载模型
print("⬇️ 正在下载 qwen3:8b...")
subprocess.run(["ollama", "pull", "qwen3:8b"], check=False)
print("✅ 模型下载完成")

⬇️ 正在安装 Ollama...
✅ Ollama 安装完成
🚀 正在后台启动 Ollama 服务...
✅ Ollama 服务已运行
⬇️ 正在下载 qwen3:8b...
✅ 模型下载完成


In [7]:
# === 模拟工具函数 ===
import re

# 知识库：模拟真实搜索结果
# 设计原则：相关信息分散在不同条目，需要多次查询才能完整回答
KNOWLEDGE_BASE = {
    # 地标建筑（用于实验1）
    "埃菲尔铁塔": "埃菲尔铁塔位于法国巴黎战神广场，高324米（含天线），建于1889年，是巴黎地标建筑。",
    "东方明珠": "东方明珠广播电视塔位于中国上海浦东，总高468米，建于1994年，是上海地标。",
    "帝国大厦": "帝国大厦位于美国纽约曼哈顿，高443米（含天线），建于1931年，曾是世界最高建筑。",
    "广州塔": "广州塔（小蛮腰）位于中国广州，总高600米，建于2010年，是中国第一高塔。",

    # 城市信息
    "巴黎": "巴黎是法国首都，人口约220万（市区），面积105平方公里，著名景点有埃菲尔铁塔、卢浮宫。",
    "上海": "上海是中国最大城市，人口约2500万，面积6340平方公里，著名地标有东方明珠、外滩。",
    "纽约": "纽约是美国最大城市，人口约840万，面积789平方公里，著名地标有自由女神像、帝国大厦。",

    # 科技人物（用于实验4的工具幻觉测试）
    "Python创始人": "Python由Guido van Rossum于1991年创建，他被称为Python之父。",
    "Guido": "Guido van Rossum，荷兰程序员，Python语言创始人，1956年出生。",

    # 公司信息（用于实验4）
    "苹果公司": "苹果公司(Apple)由乔布斯、沃兹尼亚克、韦恩于1976年创立，总部在加州库比蒂诺。",
    "特斯拉": "特斯拉(Tesla)由马斯克等人于2003年创立，总部在德州奥斯汀，主营电动汽车。",

    # ========== 实验5专用：信息分散设计 ==========
    # 设计：问"X公司CEO是谁？公司总部在哪？成立于哪年？"
    # 答案分散在3个不同条目，必须查3次才能完整回答

    "星云科技": "星云科技(Nebula Tech)是一家人工智能公司，成立于2019年，专注于大模型研发。",
    "星云科技CEO": "星云科技的CEO是张明远，他毕业于清华大学计算机系。",
    "星云科技总部": "星云科技总部位于深圳南山区科技园，占地面积约5000平方米。",
}

# 上次搜索结果缓存（用于 Lookup）
_last_search_result = {"query": "", "result": ""}

def tool_search(query):
    """搜索工具：查询知识库"""
    global _last_search_result
    query = query.strip().strip('"\'')

    # 精确匹配
    for key in KNOWLEDGE_BASE:
        if query.lower() in key.lower() or key.lower() in query.lower():
            result = KNOWLEDGE_BASE[key]
            _last_search_result = {"query": query, "result": result}
            return result

    # 模糊匹配：检查 query 中的关键词
    for key in KNOWLEDGE_BASE:
        if any(word in query for word in key.split()):
            result = KNOWLEDGE_BASE[key]
            _last_search_result = {"query": query, "result": result}
            return result

    _last_search_result = {"query": query, "result": ""}
    return f"未找到关于'{query}'的信息。请尝试其他关键词。"


def tool_calculator(expression):
    """计算器工具：执行数学计算"""
    expression = expression.strip().strip('"\'')
    # 清理表达式，只保留数字和运算符
    clean_expr = re.sub(r'[^0-9+\-*/().%\s]', '', expression)

    if not clean_expr:
        return f"计算错误：无法解析表达式 '{expression}'"

    try:
        result = eval(clean_expr, {"__builtins__": {}}, {})
        return f"{clean_expr} = {result}"
    except Exception as e:
        return f"计算错误：{str(e)}"


def tool_lookup(keyword):
    """查找工具：在上次搜索结果中查找关键词"""
    keyword = keyword.strip().strip('"\'')

    if not _last_search_result["result"]:
        return "错误：没有可供查找的搜索结果，请先使用 Search。"

    if keyword.lower() in _last_search_result["result"].lower():
        sentences = _last_search_result["result"].split("，")
        for s in sentences:
            if keyword.lower() in s.lower():
                return f"在上次搜索'{_last_search_result['query']}'的结果中找到：{s}"
        return f"找到关键词'{keyword}'，上下文：{_last_search_result['result'][:100]}"

    return f"在上次搜索结果中未找到'{keyword}'。上次搜索：{_last_search_result['query']}"


# 工具注册表
TOOLS = {
    "Search": tool_search,
    "Calculator": tool_calculator,
    "Lookup": tool_lookup,
}

print("✅ 工具函数加载完成")
print(f"📦 知识库条目: {len(KNOWLEDGE_BASE)} 条")
print(f"🔧 可用工具: {list(TOOLS.keys())}")

✅ 工具函数加载完成
📦 知识库条目: 14 条
🔧 可用工具: ['Search', 'Calculator', 'Lookup']


In [8]:
# === ReAct 执行引擎（带失败模式检测） ===

OLLAMA_URL = "http://localhost:11434"

def react_agent(question, max_iterations=6, verbose=True,
                detect_loop=True, loop_threshold=2,
                min_thought_length=5):
    """
    ReAct 执行引擎

    参数:
        question: 用户问题
        max_iterations: 最大迭代次数
        detect_loop: 是否检测 Action Loop
        loop_threshold: 连续重复多少次视为 Loop
        min_thought_length: Thought 最小词数（低于则警告碎片化）

    返回:
        dict: {
            'answer': 最终答案,
            'success': 是否成功,
            'history': 执行历史,
            'failure_mode': 失败模式（如有）,
            'stats': 统计信息
        }
    """

    system_prompt = """你是一个问题解决助手，使用 ReAct 模式（推理+行动）解决问题。

【可用工具】仅限以下 3 个：
1. Search[query] - 搜索知识库，query 是搜索关键词
2. Calculator[expression] - 数学计算，expression 是算式如 "324-468"
3. Lookup[keyword] - 在上次搜索结果中查找关键词

【输出格式】每轮严格按此格式：
Thought: [分析当前状态，决定下一步，5-20词]
Action: [工具名，只能是 Search/Calculator/Lookup]
Action Input: [工具参数]

【终止格式】确认已解决问题后：
Thought: [总结推理过程]
Final Answer: [完整答案]

【规则】
- 每次只输出一个 Thought-Action 组合
- 若连续 2 次搜索返回相同结果，应尝试其他方法或给出答案
- Final Answer 必须完整回答原始问题的所有部分"""

    history = []
    context = f"Question: {question}\n"
    recent_actions = []  # 用于检测 Action Loop
    stats = {
        'iterations': 0,
        'actions': [],
        'thought_lengths': [],
        'tool_errors': 0,
        'loop_warnings': 0
    }
    failure_mode = None

    for iteration in range(max_iterations):
        stats['iterations'] = iteration + 1

        if verbose:
            print(f"\n{'─'*60}")
            print(f"📍 第 {iteration + 1} 轮")
            print('─'*60)

        # 调用模型
        try:
            response = requests.post(
                f"{OLLAMA_URL}/api/chat",
                json={
                    "model": "qwen3:8b",
                    "messages": [
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": context}
                    ],
                    "stream": False,
                    "think": False  # 关闭内置思考
                },
                timeout=60
            )
            response.raise_for_status()
            model_output = response.json()["message"]["content"].strip()
        except Exception as e:
            if verbose:
                print(f"❌ API 调用失败: {e}")
            failure_mode = "API_ERROR"
            break

        if verbose:
            print(f"🤖 模型输出:\n{model_output}\n")

        history.append({
            "iteration": iteration + 1,
            "output": model_output,
            "context_length": len(context)
        })

        # === 检查 Final Answer ===
        if "Final Answer:" in model_output:
            match = re.search(r"Final Answer:\s*(.+)", model_output, re.DOTALL)
            if match:
                answer = match.group(1).strip()
                # 检测过早终止：迭代次数过少
                if iteration == 0:
                    failure_mode = "PREMATURE_STOP"
                    if verbose:
                        print(f"⚠️ 警告：第1轮就给出答案，可能过早终止")

                return {
                    'answer': answer,
                    'success': True,
                    'history': history,
                    'failure_mode': failure_mode,
                    'stats': stats
                }

        # === 提取 Thought ===
        thought_match = re.search(r"Thought:\s*(.+?)(?=Action:|Final Answer:|$)", model_output, re.DOTALL)
        thought = thought_match.group(1).strip() if thought_match else ""
        thought_words = len(thought.split())
        stats['thought_lengths'].append(thought_words)

        # 检测 Thought 碎片化
        if thought_words < min_thought_length:
            if verbose:
                print(f"⚠️ Thought 碎片化：仅 {thought_words} 词")

        # === 提取 Action ===
        action_match = re.search(r"Action:\s*(\w+)", model_output)
        if not action_match:
            if verbose:
                print("⚠️ 未找到有效 Action")
            failure_mode = "NO_ACTION"
            break

        action = action_match.group(1)
        stats['actions'].append(action)

        # === 检测工具幻觉 ===
        if action not in TOOLS:
            stats['tool_errors'] += 1
            observation = f"❌ 工具错误：'{action}' 不存在。仅可使用: Search, Calculator, Lookup"
            if verbose:
                print(f"🚫 工具幻觉：{action}")
            failure_mode = "TOOL_HALLUCINATION"
            context += f"\n{model_output}\nObservation: {observation}\n"
            continue

        # === 提取 Action Input ===
        input_match = re.search(r"Action Input:\s*(.+?)(?=\n|$)", model_output, re.DOTALL)
        action_input = input_match.group(1).strip() if input_match else ""

        # === 检测 Action Loop ===
        current_action = f"{action}:{action_input}"
        recent_actions.append(current_action)

        if detect_loop and len(recent_actions) >= loop_threshold:
            # 检查最近 N 次是否完全相同
            recent = recent_actions[-loop_threshold:]
            if len(set(recent)) == 1:
                stats['loop_warnings'] += 1
                if verbose:
                    print(f"⚠️ Action Loop 检测：连续 {loop_threshold} 次相同操作")
                if stats['loop_warnings'] >= 2:
                    failure_mode = "ACTION_LOOP"
                    # 注入提示打破循环
                    observation = f"⚠️ 系统提示：你已连续执行相同操作 {loop_threshold} 次，请尝试其他方法或直接给出答案。"
                    context += f"\n{model_output}\nObservation: {observation}\n"
                    continue

        # === 执行工具 ===
        if verbose:
            print(f"💭 Thought: {thought}")
            print(f"🔧 Action: {action}[{action_input}]")

        observation = TOOLS[action](action_input)

        if verbose:
            print(f"👁️ Observation: {observation}")

        context += f"\n{model_output}\nObservation: {observation}\n"

    # 达到最大迭代
    if failure_mode is None:
        failure_mode = "MAX_ITERATIONS"

    if verbose:
        print(f"\n⚠️ 执行终止，失败模式: {failure_mode}")

    return {
        'answer': None,
        'success': False,
        'history': history,
        'failure_mode': failure_mode,
        'stats': stats
    }


def print_react_result(result):
    """打印 ReAct 执行结果摘要"""
    print("\n" + "="*60)
    print("📊 执行结果")
    print("="*60)
    print(f"成功: {'✅' if result['success'] else '❌'}")
    print(f"迭代次数: {result['stats']['iterations']}")
    print(f"Action 序列: {' → '.join(result['stats']['actions'])}")

    if result['answer']:
        print(f"最终答案: {result['answer']}")

    if result['failure_mode']:
        mode_names = {
            'ACTION_LOOP': 'Action Loop（重复调用）',
            'TOOL_HALLUCINATION': '工具幻觉（调用不存在的工具）',
            'PREMATURE_STOP': '过早终止（信息不完整）',
            'MAX_ITERATIONS': '达到最大迭代',
            'NO_ACTION': '无有效 Action',
            'API_ERROR': 'API 调用错误'
        }
        print(f"失败模式: {mode_names.get(result['failure_mode'], result['failure_mode'])}")

    avg_thought = sum(result['stats']['thought_lengths']) / max(len(result['stats']['thought_lengths']), 1)
    print(f"平均 Thought 长度: {avg_thought:.1f} 词")

    if result['stats']['tool_errors'] > 0:
        print(f"工具错误次数: {result['stats']['tool_errors']}")


print("✅ ReAct 执行引擎加载完成（含失败模式检测）")

✅ ReAct 执行引擎加载完成（含失败模式检测）


## ✅ 实验 1：成功的 ReAct 流程

**问题**：埃菲尔铁塔和东方明珠哪个更高？高多少米？

**预期流程**：
1. Thought → 需要查两座塔的高度
2. Action: Search[埃菲尔铁塔] → 324米
3. Action: Search[东方明珠] → 468米
4. Action: Calculator[468-324] → 144
5. Final Answer: 东方明珠更高，高144米

In [9]:
# ============================================================
# 实验 1：成功的 ReAct 流程
# ============================================================
# 设计思路：这个问题需要 2-3 步完成
#   1. Search 查询埃菲尔铁塔高度 → 得到 324 米
#   2. Calculator 计算差值 → 324 - 468 = -144
#   3. Final Answer 解释结果
#
# 这能完整展示 Thought → Action → Observation 循环

question_success = "埃菲尔铁塔和东方明珠哪个更高？高多少米？"

print("="*60)
print("✅ 实验 1：成功的 ReAct 流程")
print("="*60)
print(f"问题: {question_success}")
print("="*60)

result = react_agent(question_success, max_iterations=5, verbose=True)
print_react_result(result)

✅ 实验 1：成功的 ReAct 流程
问题: 埃菲尔铁塔和东方明珠哪个更高？高多少米？

────────────────────────────────────────────────────────────
📍 第 1 轮
────────────────────────────────────────────────────────────
🤖 模型输出:
Thought: 需要查找埃菲尔铁塔和东方明珠的高度数据。
Action: Search
Action Input: 埃菲尔铁塔高度 东方明珠高度

⚠️ Thought 碎片化：仅 1 词
💭 Thought: 需要查找埃菲尔铁塔和东方明珠的高度数据。
🔧 Action: Search[埃菲尔铁塔高度 东方明珠高度]
👁️ Observation: 埃菲尔铁塔位于法国巴黎战神广场，高324米（含天线），建于1889年，是巴黎地标建筑。

────────────────────────────────────────────────────────────
📍 第 2 轮
────────────────────────────────────────────────────────────
🤖 模型输出:
Thought: 需要查找东方明珠的高度数据。
Action: Search
Action Input: 东方明珠高度

⚠️ Thought 碎片化：仅 1 词
💭 Thought: 需要查找东方明珠的高度数据。
🔧 Action: Search[东方明珠高度]
👁️ Observation: 东方明珠广播电视塔位于中国上海浦东，总高468米，建于1994年，是上海地标。

────────────────────────────────────────────────────────────
📍 第 3 轮
────────────────────────────────────────────────────────────
🤖 模型输出:
Thought: 已获取两者高度数据，可计算差值。
Action: Calculator
Action Input: 468-324

Thought: 计算结果确认东方明珠比埃菲尔铁塔高。
Final Answer: 东方明珠比埃菲尔铁塔高144米，埃菲

## ❌ 实验 2：失败模式 - Action Loop

**现象**：反复调用同一工具，参数几乎不变，无法推进

```
Action: Search["诺贝尔奖"]
Observation: 未找到
Action: Search["诺贝尔奖得主"]  ← 重复搜索
Observation: 未找到
...（循环）
```

**根因**：
- 知识库无相关信息，但模型不知道何时停止
- 缺少"信息不足时应终止"的判断逻辑

**修复**：在引擎中检测连续重复，自动注入终止提示

In [10]:
# ============================================================
# 实验 2：失败模式 - Action Loop（重复调用循环）
# ============================================================
# 设计思路：问一个知识库中不存在的信息
#   模型会反复 Search 尝试找答案，触发 Action Loop 检测
#   引擎会在连续 2 次相同操作后发出警告

question_loop = "2024年诺贝尔物理学奖得主是谁？"

print("="*60)
print("❌ 实验 2：Action Loop 失败模式")
print("="*60)
print(f"问题: {question_loop}")
print("设计意图: 知识库无此信息，模型可能反复搜索")
print("="*60)

result = react_agent(
    question_loop,
    max_iterations=5,
    verbose=True,
    detect_loop=True,
    loop_threshold=2
)
print_react_result(result)

# 分析 Action 序列
if result['stats']['actions']:
    print("\n📈 Action 序列分析:")
    actions = result['stats']['actions']
    for i, action in enumerate(actions, 1):
        repeat = "⚠️ 重复" if i > 1 and action == actions[i-2] else ""
        print(f"  {i}. {action} {repeat}")

❌ 实验 2：Action Loop 失败模式
问题: 2024年诺贝尔物理学奖得主是谁？
设计意图: 知识库无此信息，模型可能反复搜索

────────────────────────────────────────────────────────────
📍 第 1 轮
────────────────────────────────────────────────────────────
🤖 模型输出:
Thought: 需要查找2024年诺贝尔物理学奖得主的信息。
Action: Search
Action Input: 2024年诺贝尔物理学奖得主

⚠️ Thought 碎片化：仅 1 词
💭 Thought: 需要查找2024年诺贝尔物理学奖得主的信息。
🔧 Action: Search[2024年诺贝尔物理学奖得主]
👁️ Observation: 未找到关于'2024年诺贝尔物理学奖得主'的信息。请尝试其他关键词。

────────────────────────────────────────────────────────────
📍 第 2 轮
────────────────────────────────────────────────────────────
🤖 模型输出:
Thought: 尝试使用更广泛的关键词，如“2024诺贝尔物理学奖”。  
Action: Search  
Action Input: 2024诺贝尔物理学奖

⚠️ Thought 碎片化：仅 1 词
💭 Thought: 尝试使用更广泛的关键词，如“2024诺贝尔物理学奖”。
🔧 Action: Search[2024诺贝尔物理学奖]
👁️ Observation: 未找到关于'2024诺贝尔物理学奖'的信息。请尝试其他关键词。

────────────────────────────────────────────────────────────
📍 第 3 轮
────────────────────────────────────────────────────────────
🤖 模型输出:
Thought: 尝试搜索“诺贝尔物理学奖 2024”以获取最新信息。  
Action: Search  
Action Input: 诺贝尔物理学奖 

## 📊 实验 3：Thought 质量分析

**好的 Thought**：
```
Thought: 要比较两座塔的高度，需要分别查询埃菲尔铁塔和东方明珠的高度数据
```

**碎片化 Thought**：
```
Thought: 搜索一下
Thought: 继续
Thought: 好的
```

**影响**：Token 浪费、推理链断裂、难以 debug

**检测标准**：Thought 少于 5 个词视为碎片化

In [11]:
# ============================================================
# 实验 3：Thought 碎片化分析
# ============================================================
# 说明：不是单独实验，而是分析前面实验的 Thought 质量
#   好的 Thought：清晰说明当前状态和下一步计划
#   碎片化 Thought：过短、无意义（如"搜索"、"好"）

def analyze_thought_quality(result, min_words=5):
    """分析 ReAct 执行中的 Thought 质量"""
    print("="*60)
    print("📊 Thought 质量分析")
    print("="*60)

    thoughts = []
    for step in result['history']:
        match = re.search(r"Thought:\s*(.+?)(?=Action:|Final Answer:|$)",
                         step['output'], re.DOTALL)
        if match:
            thoughts.append(match.group(1).strip())

    if not thoughts:
        print("未找到 Thought")
        return

    fragmented = 0
    for i, thought in enumerate(thoughts, 1):
        words = len(thought.split())
        status = "⚠️ 碎片化" if words < min_words else "✅"
        print(f"\nThought {i} ({words}词) {status}")
        print(f"  「{thought[:80]}{'...' if len(thought) > 80 else ''}」")
        if words < min_words:
            fragmented += 1

    rate = fragmented / len(thoughts) * 100
    print(f"\n碎片化率: {rate:.0f}% ({fragmented}/{len(thoughts)})")
    if rate > 30:
        print("⚠️ 建议：在 Prompt 中要求 Thought 必须包含「当前状态」和「下一步计划」")

# 分析实验 1 的结果
if 'result' in dir() and result:
    analyze_thought_quality(result)
else:
    print("请先运行实验 1 或实验 2")

📊 Thought 质量分析

Thought 1 (1词) ⚠️ 碎片化
  「需要查找2024年诺贝尔物理学奖得主的信息。」

Thought 2 (1词) ⚠️ 碎片化
  「尝试使用更广泛的关键词，如“2024诺贝尔物理学奖”。」

Thought 3 (2词) ⚠️ 碎片化
  「尝试搜索“诺贝尔物理学奖 2024”以获取最新信息。」

Thought 4 (1词) ⚠️ 碎片化
  「2024年的诺贝尔奖尚未公布，因此无法获取得主信息。」

碎片化率: 100% (4/4)
⚠️ 建议：在 Prompt 中要求 Thought 必须包含「当前状态」和「下一步计划」


## ❌ 实验 4：失败模式 - 工具幻觉

**现象**：调用不存在的工具名

```
Action: GoogleSearch["苹果股价"]  ← 不存在！
Action: WebBrowser["url"]        ← 不存在！
Action: StockAPI["AAPL"]         ← 不存在！
```

**实际可用**：Search, Calculator, Lookup

**根因**：模型训练数据中见过类似工具名，产生"幻觉"

**修复**：
1. Prompt 中明确列出工具清单
2. 引擎层验证工具名，返回错误提示

In [12]:
# ============================================================
# 实验 4：工具幻觉（Tool Hallucination）
# ============================================================
# 设计思路：问一个涉及网络/数据库的问题
#   模型可能幻想出 WebSearch、Database 等不存在的工具
#   引擎会拦截并返回错误提示

question_hallucination = "帮我在网上查一下苹果公司最新的股票价格"

print("="*60)
print("❌ 实验 4：工具幻觉失败模式")
print("="*60)
print(f"问题: {question_hallucination}")
print("设计意图: 诱导模型调用 WebSearch/StockAPI 等不存在的工具")
print(f"实际可用工具: {list(TOOLS.keys())}")
print("="*60)

result_hallucination = react_agent(
    question_hallucination,
    max_iterations=4,
    verbose=True
)
print_react_result(result_hallucination)

# 统计工具错误
if result_hallucination['stats']['tool_errors'] > 0:
    print(f"\n🚫 检测到 {result_hallucination['stats']['tool_errors']} 次工具幻觉")
    print("常见幻觉工具: WebSearch, GoogleSearch, StockAPI, Database, Browser")

❌ 实验 4：工具幻觉失败模式
问题: 帮我在网上查一下苹果公司最新的股票价格
设计意图: 诱导模型调用 WebSearch/StockAPI 等不存在的工具
实际可用工具: ['Search', 'Calculator', 'Lookup']

────────────────────────────────────────────────────────────
📍 第 1 轮
────────────────────────────────────────────────────────────
🤖 模型输出:
Thought: 需要查找苹果公司最新的股票价格，先进行网络搜索。
Action: Search
Action Input: 苹果公司 最新股票价格

⚠️ Thought 碎片化：仅 1 词
💭 Thought: 需要查找苹果公司最新的股票价格，先进行网络搜索。
🔧 Action: Search[苹果公司 最新股票价格]
👁️ Observation: 苹果公司(Apple)由乔布斯、沃兹尼亚克、韦恩于1976年创立，总部在加州库比蒂诺。

────────────────────────────────────────────────────────────
📍 第 2 轮
────────────────────────────────────────────────────────────
🤖 模型输出:
Thought: 搜索结果未提供股票价格，需尝试其他方法或直接给出答案。
Final Answer: 我无法提供实时股票价格，建议您查看财经新闻网站或股票交易平台获取最新信息。


📊 执行结果
成功: ✅
迭代次数: 2
Action 序列: Search
最终答案: 我无法提供实时股票价格，建议您查看财经新闻网站或股票交易平台获取最新信息。
平均 Thought 长度: 1.0 词


## ❌ 实验 5：失败模式 - 过早终止

**设计思路**：问一个需要查询**多个独立条目**才能完整回答的问题

知识库设计：
- `"星云科技"` → 只有成立年份(2019)
- `"星云科技CEO"` → 只有CEO姓名(张明远)  
- `"星云科技总部"` → 只有总部地址(深圳)

**问题**：星云科技的CEO是谁？总部在哪？成立于哪年？

**过早终止表现**：
```
Search["星云科技"] → 成立于2019年
Thought: 我知道答案了
Final Answer: 星云科技成立于2019年  ← 漏了CEO和总部！
```

**正确流程**：需要3次Search才能完整回答

In [13]:
# ============================================================
# 实验 5：过早终止（Premature Stop）
# ============================================================
# 设计思路：
#   知识库中"星云科技"的信息故意分散在3个条目：
#   - "星云科技" → 成立年份
#   - "星云科技CEO" → CEO姓名
#   - "星云科技总部" → 总部地址
#
#   模型只查一次就回答 = 过早终止（信息不完整）
#   正确做法 = 查3次，每次获取一个信息点

question_premature = "星云科技的CEO是谁？公司总部在哪个城市？成立于哪一年？"

print("="*60)
print("❌ 实验 5：过早终止失败模式")
print("="*60)
print(f"问题: {question_premature}")
print("\n📌 知识库设计（信息分散）:")
print("  - Search[星云科技] → 只能得到成立年份")
print("  - Search[星云科技CEO] → 只能得到CEO姓名")
print("  - Search[星云科技总部] → 只能得到总部地址")
print("\n⚠️ 如果模型只查1-2次就回答，说明过早终止")
print("="*60)

result_premature = react_agent(
    question_premature,
    max_iterations=5,
    verbose=True
)
print_react_result(result_premature)

# 检查答案完整性
if result_premature['answer']:
    answer = result_premature['answer']
    print("\n📋 答案完整性检查（3个必答项）:")
    checks = [
        ("CEO姓名", "张明远" in answer),
        ("总部城市", "深圳" in answer),
        ("成立年份", "2019" in answer),
    ]
    passed_count = sum(1 for _, passed in checks if passed)

    for item, passed in checks:
        print(f"  {item}: {'✅' if passed else '❌ 缺失'}")

    print(f"\n完整度: {passed_count}/3")

    if passed_count < 3:
        print("⚠️ 答案不完整！发生了过早终止")
        print(f"   模型只进行了 {len(result_premature['stats']['actions'])} 次 Action")
        print("   正确做法应该是查询3次：星云科技、星云科技CEO、星云科技总部")
    else:
        search_count = result_premature['stats']['actions'].count('Search')
        print(f"✅ 答案完整，模型进行了 {search_count} 次 Search")

❌ 实验 5：过早终止失败模式
问题: 星云科技的CEO是谁？公司总部在哪个城市？成立于哪一年？

📌 知识库设计（信息分散）:
  - Search[星云科技] → 只能得到成立年份
  - Search[星云科技CEO] → 只能得到CEO姓名
  - Search[星云科技总部] → 只能得到总部地址

⚠️ 如果模型只查1-2次就回答，说明过早终止

────────────────────────────────────────────────────────────
📍 第 1 轮
────────────────────────────────────────────────────────────
🤖 模型输出:
Thought: 需要搜索星云科技的CEO、总部城市和成立年份。
Action: Search
Action Input: 星云科技 CEO 总部 成立年份

⚠️ Thought 碎片化：仅 1 词
💭 Thought: 需要搜索星云科技的CEO、总部城市和成立年份。
🔧 Action: Search[星云科技 CEO 总部 成立年份]
👁️ Observation: 星云科技(Nebula Tech)是一家人工智能公司，成立于2019年，专注于大模型研发。

────────────────────────────────────────────────────────────
📍 第 2 轮
────────────────────────────────────────────────────────────
🤖 模型输出:
Thought: 需要进一步搜索星云科技的CEO和总部城市。
Action: Search
Action Input: 星云科技 CEO 总部城市

⚠️ Thought 碎片化：仅 1 词
💭 Thought: 需要进一步搜索星云科技的CEO和总部城市。
🔧 Action: Search[星云科技 CEO 总部城市]
👁️ Observation: 星云科技(Nebula Tech)是一家人工智能公司，成立于2019年，专注于大模型研发。

────────────────────────────────────────────────────────────
📍 第 3 轮
────────────────

## 📊 实验总结

### ReAct 核心流程

```
Thought → Action → Observation → Thought → ... → Final Answer
 (推理)    (行动)    (观察)       (推理)           (答案)
```

### 四大失败模式

| 模式 | 表现 | 检测 | 修复 |
|-----|------|------|------|
| **Action Loop** | 重复调用同一工具 | 连续N次相同Action | 设置max_iterations + 注入终止提示 |
| **Thought碎片化** | 思考过短无意义 | 词数<5 | Prompt要求详细思考 |
| **工具幻觉** | 调用不存在的工具 | 工具名验证 | 明确列出工具清单 + 返回错误 |
| **过早终止** | 信息不全就回答 | 迭代次数过少/答案不完整 | Prompt添加检查清单 |

### ReAct vs CoT

| 维度 | CoT | ReAct |
|-----|-----|-------|
| 能力 | 纯推理 | 推理 + 工具调用 |
| 外部信息 | ❌ | ✅ 通过工具获取 |
| 适用场景 | 数学推理、逻辑分析 | 需要查询、计算的任务 |
| 主要失败 | 推理错误 | Loop、幻觉、过早终止 |

### 最佳实践

```python
# 1. 清晰定义工具（防止幻觉）
"""可用工具【仅限以下3个】:
- Search[query] - 搜索
- Calculator[expr] - 计算  
- Lookup[keyword] - 查找"""

# 2. 设置迭代上限（防止Loop）
max_iterations = 5

# 3. 要求详细Thought（防止碎片化）
"Thought必须包含：当前状态 + 下一步计划"

# 4. 添加完整性检查（防止过早终止）
"Final Answer前检查：是否回答了问题的所有部分？"
```

**ReAct 是现代 AI Agent（如 LangChain、AutoGPT）的核心模式。**